### Spencer Investment Strategy Analysis
Objective: run a simulation to measure the returns of investing in the top 100 market cap stocks, but only purchasing after they have dropped to x% below their 52 week high. And then selling those stocks once they surpass y% of my purchase price. Run simulation for z years, re-evaluating stocks every certain number of days. Compare to S&P 500 returns.

Constraints: buy a specific amount of stock every time a buy signal is triggered. Allow the purchase of fractional shares. There is no limit to the amount of money that may be invested at any given time.


### User Inputs

In [1]:
# Buy and sell percentages
buy_after_52_week_high_percentage_drop = 0.2
sell_after_percentage_rise_from_purchase_price = 0.1

# Number of years to look back
lookback_years = 5

# How many days between each buy/sell check
# 1 means buy/sell every day for lookback_years
buy_sell_interval_in_days = 1

# How much money to invest when a buy signal is triggered (in USD)
investment_amount_per_buy_signal = 1000

# Logging parameters
output_portfolio = False
output_transactions = True
output_outstanding_investment = False


### Imports

In [2]:
# Import the yfinance library to get historical stock price data
%pip install yfinance pandas numpy --user

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


In [3]:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Ignore this specific warning associatd with the yfinance library: https://github.com/ranaroussi/yfinance/issues/1837
import warnings
warnings.filterwarnings("ignore", message="The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.", category=FutureWarning, module="yfinance.utils")

In [4]:
# Calculate the start date. End date of the simulation is 2 days ago. 
start_date = (datetime.now() - timedelta(days=365 * lookback_years + 2)).strftime('%Y-%m-%d')
end_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')

In [5]:
# Hard-coded list of stocks in the S&P 100 as of September 18, 2023
# TODO: Automatically fetch this list for each day of the simulation
sp100_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
# sp100_stocks = ['AAPL ', 'ABBV ', 'ABT ', 'ACN ', 'ADBE ', 'AIG ', 'AMD ', 'AMGN ', 'AMT ', 'AMZN ', 'AVGO ', 'AXP ', 'BA ', 'BAC ', 'BK ', 'BKNG ', 'BLK ', 'BMY ', 'BRK.B ', 'C ', 'CAT ', 'CHTR ', 'CL ', 'CMCSA ', 'COF ', 'COP ', 'COST ', 'CRM ', 'CSCO ', 'CVS ', 'CVX ', 'DE ', 'DHR ', 'DIS ', 'DOW ', 'DUK ', 'EMR ', 'EXC ', 'F ', 'FDX ', 'GD ', 'GE ', 'GILD ', 'GM ', 'GOOG ', 'GOOGL ', 'GS ', 'HD ', 'HON ', 'IBM ', 'INTC ', 'JNJ ', 'JPM ', 'KHC ', 'KO ', 'LIN ', 'LLY ', 'LMT ', 'LOW ', 'MA ', 'MCD ', 'MDLZ ', 'MDT ', 'MET ', 'META ', 'MMM ', 'MO ', 'MRK ', 'MS ', 'MSFT ', 'NEE ', 'NFLX ', 'NKE ', 'NVDA ', 'ORCL ', 'PEP ', 'PFE ', 'PG ', 'PM ', 'PYPL ', 'QCOM ', 'RTX ', 'SBUX ', 'SCHW ', 'SO ', 'SPG ', 'T ', 'TGT ', 'TMO ', 'TMUS ', 'TSLA ', 'TXN ', 'UNH ', 'UNP ', 'UPS ', 'USB ', 'V ', 'VZ ', 'WFC ', 'WMT ', 'XOM ']

In [6]:
# DataFrames for transactions, current portfolio, and outstanding investment
transactions = pd.DataFrame(columns=['Date', 'Ticker', 'Action', 'Buy_or_Sell_Price', 'Number_of_Shares', 'Cash_Flow', 'Entry_ID'])
portfolio = pd.DataFrame(columns=['Entry_ID', 'Ticker', 'Shares', 'Purchase_Price', 'Book_Value'])
outstanding_investment = pd.DataFrame(columns=['Date', 'Outstanding_Invested_Amount'])

In [7]:
# Function to simulate buying
def buy_stock(date, ticker, close_price):
    global transactions, portfolio

    shares_bought = investment_amount_per_buy_signal / close_price

    # Generate a unique Entry_ID using a high-resolution timestamp
    entry_id = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

    # Update transactions DataFrame
    new_transaction_entry = pd.DataFrame({
        'Date': [date],
        'Ticker': [ticker],
        'Action': ['Buy'],
        'Buy_or_Sell_Price': [close_price],
        'Number_of_Shares': [shares_bought],
        'Cash_Flow': [-investment_amount_per_buy_signal],
        'Entry_ID': [entry_id]
    })

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        transactions = pd.concat([transactions, new_transaction_entry], ignore_index=True)

    # Update portfolio DataFrame
    new_portfolio_entry = pd.DataFrame({
        'Entry_ID': [entry_id],
        'Ticker': [ticker], 
        'Shares': [shares_bought], 
        'Purchase_Price': [close_price], 
        'Book_Value': [shares_bought * close_price]
    })

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        portfolio = pd.concat([portfolio, new_portfolio_entry], ignore_index=True)

    # Update the outstanding investment
    update_outstanding_investment(date, investment_amount_per_buy_signal)

    # Logging the buy
    if output_transactions:
        print(f"Buy: {date.strftime('%Y-%m-%d')} - {ticker}, Shares: {shares_bought:.4f}, Price: {close_price}, Value: {investment_amount_per_buy_signal:.2f}")


# Function to simulate selling
def sell_stock(date, entry_id, sell_price):
    global transactions, portfolio

    # Locate the transaction to be sold using Entry_ID
    transaction_index = portfolio[portfolio['Entry_ID'] == entry_id].index
    if not transaction_index.empty:
        index = transaction_index[0]
        ticker = portfolio.at[index, 'Ticker']
        shares_sold = portfolio.at[index, 'Shares']
        sold_value = shares_sold * sell_price

        # Update transactions DataFrame with the sale information
        new_sale_entry = pd.DataFrame({
            'Date': [date],
            'Ticker': [ticker],
            'Action': ['Sell'],
            'Buy_or_Sell_Price': [sell_price],
            'Number_of_Shares': [shares_sold],
            'Cash_Flow': [sold_value],
            'Entry_ID': [entry_id]
        })
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", FutureWarning)
            transactions = pd.concat([transactions, new_sale_entry], ignore_index=True)

        # Update outstanding investment
        update_outstanding_investment(date, -investment_amount_per_buy_signal)

        # Remove the sold stock from the portfolio
        portfolio.drop(index, inplace=True)
        portfolio.reset_index(drop=True, inplace=True)

        # Logging the sale
        if output_transactions:
            print(f"Sell: {date.strftime('%Y-%m-%d')} - {ticker}, Shares: {shares_sold:.4f}, Price: {sell_price}, Value: {sold_value:.2f}")
    else:
        print(f"ERROR: Entry ID {entry_id} not found in the portfolio.")

# Update outstanding investment
def update_outstanding_investment(date, amount):
    if outstanding_investment.empty or date not in outstanding_investment['Date'].values:
        if not outstanding_investment.empty:
            # If there's a previous entry, carry over the last outstanding amount and add the current amount
            last_outstanding_amount = outstanding_investment.iloc[-1]['Outstanding_Invested_Amount']
            new_outstanding_amount = last_outstanding_amount + amount
        else:
            # If there are no previous entries, this is the first transaction
            new_outstanding_amount = amount
        outstanding_investment.loc[len(outstanding_investment)] = [date, new_outstanding_amount]
    else:
        # Update the existing entry for the date
        last_index = outstanding_investment[outstanding_investment['Date'] == date].index[-1]
        outstanding_investment.at[last_index, 'Outstanding_Invested_Amount'] += amount


In [8]:
# Load historical data for each ticker
historical_data = {ticker: yf.download(ticker, start=start_date, end=end_date, progress=False) for ticker in sp100_stocks}
for ticker, data in historical_data.items():
    # The choice of 252 days is based on the typical number of trading days in a year for the U.S. stock market, which is commonly used as an approximation for a 52-week period.
    # The calculated 52-week high at any given row in the DataFrame represents the highest closing price over the preceding 252 trading days from that row
    data['52_Week_High'] = data['Close'].rolling(window=252).max()

In [9]:
# Simulation loop
current_date = datetime.strptime(start_date, '%Y-%m-%d')
end_simulation_date = datetime.strptime(end_date, '%Y-%m-%d')

while current_date <= end_simulation_date:
    day_has_transactions = False

    # For each day in the simulation, check if we should buy or sell each stock
    for ticker, data in historical_data.items():
        if current_date.strftime('%Y-%m-%d') in data.index:
            row = data.loc[current_date.strftime('%Y-%m-%d')]

            # Buy logic
            if row['Close'] <= row['52_Week_High'] * (1 - buy_after_52_week_high_percentage_drop):
                buy_stock(current_date, ticker, row['Close'])
                day_has_transactions = True

            # Sell logic
            to_sell = []
            for index, stock in portfolio[portfolio['Ticker'] == ticker].iterrows():
                if row['Close'] >= stock['Purchase_Price'] * (1 + sell_after_percentage_rise_from_purchase_price):
                    # Add an entry to the DataFrame to_sell containing the stock to be sold. Uniquely
                    # identify the stock from the portfolio using Entry_ID
                    to_sell.append((stock['Entry_ID'], row['Close']))
            
            # Sell all stocks that meet the sell criteria
            for entry_id, sell_price in to_sell:
                sell_stock(current_date, entry_id, sell_price)
                day_has_transactions = True


    if output_portfolio:
        # Print the current portfolio:
        portfolio_without_entry_id = portfolio.drop('Entry_ID', axis=1)
        print(f"\nCurrent Portfolio as of {current_date.strftime('%Y-%m-%d')}:")
        print(portfolio_without_entry_id)
        print("\n")
    
    # If no transactions occurred on this day, carry over the last outstanding investment amount
    if not day_has_transactions:
        update_outstanding_investment(current_date, 0)

    if output_outstanding_investment:
        # Print statements for current date and outstanding investment
        if not outstanding_investment.empty:
            current_outstanding_investment = outstanding_investment[outstanding_investment['Date'] == current_date.strftime('%Y-%m-%d')]['Outstanding_Invested_Amount'].iloc[-1]
            print("Date:", current_date.strftime('%Y-%m-%d'), "Outstanding Investment: $ ", "{:,.2f}".format(current_outstanding_investment))
        else:
            print("Date:", current_date.strftime('%Y-%m-%d'), "Outstanding Investment: $ 0.00")


    current_date += timedelta(days=buy_sell_interval_in_days)

Buy: 2020-03-09 - MSFT, Shares: 6.6392, Price: 150.6199951171875, Value: 1000.00
Buy: 2020-03-09 - GOOGL, Shares: 16.4502, Price: 60.78950119018555, Value: 1000.00
Buy: 2020-03-11 - GOOGL, Shares: 16.5166, Price: 60.54499816894531, Value: 1000.00
Buy: 2020-03-12 - AAPL, Shares: 16.1141, Price: 62.057498931884766, Value: 1000.00
Buy: 2020-03-12 - MSFT, Shares: 7.1911, Price: 139.05999755859375, Value: 1000.00
Buy: 2020-03-12 - GOOGL, Shares: 17.9929, Price: 55.57749938964844, Value: 1000.00
Buy: 2020-03-12 - AMZN, Shares: 11.9288, Price: 83.83049774169922, Value: 1000.00
Sell: 2020-03-13 - AAPL, Shares: 16.1141, Price: 69.49250030517578, Value: 1119.81
Sell: 2020-03-13 - MSFT, Shares: 7.1911, Price: 158.8300018310547, Value: 1142.17
Buy: 2020-03-13 - GOOGL, Shares: 16.4708, Price: 60.7135009765625, Value: 1000.00
Buy: 2020-03-16 - AAPL, Shares: 16.5146, Price: 60.5525016784668, Value: 1000.00
Buy: 2020-03-16 - MSFT, Shares: 7.3844, Price: 135.4199981689453, Value: 1000.00
Buy: 2020-03-1

In [10]:
# Convert end_date to datetime for comparison
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

# End of simulation: Sell all outstanding stocks
for index, stock in portfolio.iterrows():
    print("\n--- End of Simulation Sell-Off ---")
    ticker = stock['Ticker']
    entry_id = stock['Entry_ID']

    # Check if we have historical data for this ticker
    if ticker in historical_data:
        # Get the data for this ticker
        data = historical_data[ticker]

        # Find the last available price before or on the end_date
        # Filter the data up to and including the end_date
        filtered_data = data[data.index <= end_date_dt]

        if not filtered_data.empty:
            # Get the last available closing price
            last_price = filtered_data['Close'].iloc[-1]
            # Sell the stock using sell_stock()
            sell_stock(end_date_dt, entry_id, last_price)
        else:
            print(f"No available closing price for {ticker} on or before {end_date}")
    else:
        print(f"No historical data found for {ticker}")

# Print outstanding investment for the last day, which should be $0 after all stock has been sold
end_simulation_date = datetime.strptime(end_date, '%Y-%m-%d')
if not outstanding_investment.empty:
    ending_outstanding_investment = outstanding_investment.iloc[-1]['Outstanding_Invested_Amount']
    print("Ending Date:", end_date, "Ending Outstanding Investment: $", "{:,.2f}".format(ending_outstanding_investment))
else:
    print("Ending Date:", end_date, "Ending Outstanding Investment: $0.00")

Ending Date: 2024-02-21 Ending Outstanding Investment: $ 0.00


In [11]:
# Calculate total investment and returns
total_invested = transactions[transactions['Action'] == 'Buy']['Cash_Flow'].sum()
total_returned = transactions[transactions['Action'] == 'Sell']['Cash_Flow'].sum()
total_gain_loss = total_returned + total_invested  # Since invested amount is negative

print("Total Cumulative Invested: $", "{:,.2f}".format(-total_invested))
print("Total Cumulative Returned: $", "{:,.2f}".format(total_returned))
print("Total Gain/Loss: $", "{:,.2f}".format(total_gain_loss))

Total Cumulative Invested: $ 844,000.00
Total Cumulative Returned: $ 940,537.99
Total Gain/Loss: $ 96,537.99


In [12]:
# Show the outstanding investment (how much cash you have used to acquire your current portfolio) per trading day
for index, row in outstanding_investment.iterrows():
    print("Date: ",row['Date'].strftime('%Y-%m-%d'), "Outstanding Investment: $ ", "{:,.2f}".format(row['Outstanding_Invested_Amount']))

Date:  2019-02-22 Outstanding Investment: $  0.00
Date:  2019-02-23 Outstanding Investment: $  0.00
Date:  2019-02-24 Outstanding Investment: $  0.00
Date:  2019-02-25 Outstanding Investment: $  0.00
Date:  2019-02-26 Outstanding Investment: $  0.00
Date:  2019-02-27 Outstanding Investment: $  0.00
Date:  2019-02-28 Outstanding Investment: $  0.00
Date:  2019-03-01 Outstanding Investment: $  0.00
Date:  2019-03-02 Outstanding Investment: $  0.00
Date:  2019-03-03 Outstanding Investment: $  0.00
Date:  2019-03-04 Outstanding Investment: $  0.00
Date:  2019-03-05 Outstanding Investment: $  0.00
Date:  2019-03-06 Outstanding Investment: $  0.00
Date:  2019-03-07 Outstanding Investment: $  0.00
Date:  2019-03-08 Outstanding Investment: $  0.00
Date:  2019-03-09 Outstanding Investment: $  0.00
Date:  2019-03-10 Outstanding Investment: $  0.00
Date:  2019-03-11 Outstanding Investment: $  0.00
Date:  2019-03-12 Outstanding Investment: $  0.00
Date:  2019-03-13 Outstanding Investment: $  0.00
